In [2]:
!pip install -q transformers datasets evaluate accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Libraries

In [19]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import pandas as pd
import evaluate
from sklearn.metrics import classification_report


In [4]:
# Load data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Convert to Hugging face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [5]:
# 3. Tokenizer
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/34080 [00:00<?, ? examples/s]

Map:   0%|          | 0/8520 [00:00<?, ? examples/s]

In [7]:
# Remove unnecessary stuff
train_dataset = train_dataset.remove_columns(["text"]).rename_column("label", "labels")
test_dataset = test_dataset.remove_columns(["text"]).rename_column("label", "labels")

train_dataset.set_format("torch")
test_dataset.set_format("torch")

## Load distilbert

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Finetune distilbert

In [9]:
# 5. Configure training
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-9-b21b6f00aa30>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cornualghost (cornualghost-ewcwdcd) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.146200,0.079897,0.975822
2,0.057500,0.086556,0.978404
3,0.024300,0.097396,0.980751


TrainOutput(global_step=6390, training_loss=0.07601205231811332, metrics={'train_runtime': 5118.4301, 'train_samples_per_second': 19.975, 'train_steps_per_second': 1.248, 'total_flos': 1.354346683858944e+16, 'train_loss': 0.07601205231811332, 'epoch': 3.0})

Save models in zip

In [10]:
!zip -r final_model.zip ./results

  adding: results/ (stored 0%)
  adding: results/checkpoint-6390/ (stored 0%)
  adding: results/checkpoint-6390/special_tokens_map.json (deflated 42%)
  adding: results/checkpoint-6390/trainer_state.json (deflated 66%)
  adding: results/checkpoint-6390/tokenizer_config.json (deflated 75%)
  adding: results/checkpoint-6390/vocab.txt (deflated 53%)
  adding: results/checkpoint-6390/training_args.bin (deflated 51%)
  adding: results/checkpoint-6390/config.json (deflated 45%)
  adding: results/checkpoint-6390/scheduler.pt (deflated 56%)
  adding: results/checkpoint-6390/tokenizer.json (deflated 71%)
  adding: results/checkpoint-6390/model.safetensors (deflated 8%)
  adding: results/checkpoint-6390/optimizer.pt (deflated 23%)
  adding: results/checkpoint-6390/rng_state.pth (deflated 25%)
  adding: results/checkpoint-4260/ (stored 0%)
  adding: results/checkpoint-4260/special_tokens_map.json (deflated 42%)
  adding: results/checkpoint-4260/trainer_state.json (deflated 63%)
  adding: results/

See details of training:
https://wandb.ai/cornualghost-ewcwdcd/distilbert_finetuned/runs/7m67nqac?nw=nwusercornualghost

## Inference

In [14]:
preds = trainer.predict(test_dataset)
y_pred = np.argmax(preds.predictions, axis=1)
y_true = preds.label_ids

print("BERT Classification Report:")
print(classification_report(y_true, y_pred, digits=3))

BERT Classification Report:
              precision    recall  f1-score   support

           0      0.980     0.987     0.984      5036
           1      0.981     0.971     0.976      3484

    accuracy                          0.981      8520
   macro avg      0.981     0.979     0.980      8520
weighted avg      0.981     0.981     0.981      8520



## Save models to Hugging Face

In [16]:
!pip install huggingface_hub


In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [26]:
from huggingface_hub import create_repo, upload_folder

model_dir = "results/checkpoint-6390"
repo_name = "tm4scam-distilbert"
create_repo(repo_name, private=False)
upload_folder(
    folder_path=model_dir,
    repo_id=f"cornualghost/{repo_name}",
    repo_type="model"
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/536M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cornualghost/tm4scam-distilbert/commit/4023a3c11ab9ba02f90a374df03299eced13cca1', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4023a3c11ab9ba02f90a374df03299eced13cca1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cornualghost/tm4scam-distilbert', endpoint='https://huggingface.co', repo_type='model', repo_id='cornualghost/tm4scam-distilbert'), pr_revision=None, pr_num=None)